In [1]:
%load_ext autoreload
%autoreload 2
import os
from huggingface_hub import login
from collections import defaultdict
from projects.wiki_experts.src.evolution.utils import get_svd_embedding

from mttl.models.modifiers.expert_containers.expert_library import LocalExpertLibrary,  HFExpertLibrary
from mttl.models.modifiers.expert_containers.library_transforms import SVDEmbeddingTransform, SVDEmbeddingTransformConfig
from huggingface_hub import login, HfApi
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity

/anaconda/envs/comp_3.9/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/anaconda/envs/comp_3.9/lib/python3.9/site-packages/torchvision/image.so: undefined symbol: _ZN3c104cuda9SetDeviceEi'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


Registering modifier...lora
Registering modifier...poly
Registering modifier...per_token_poly
Registering modifier...skilled
Registering modifier...kv_adapter
Registering modifier...poly_kv_adapter
Registering modifier...prompt_tuning
Registering modifier...poly_prompt_tuning
Registering modifier...hard_prompt
Registering multi-expert selector...poly_router
Registering multi-expert selector...moe_rkhs_router
Registering multi-expert selector...zero_router
Registering multi-expert selector...zero_per_token_router
Registering multi-expert selector...poly_router_dir
Registering multi-expert selector...info_selector
Registering multi-expert selector...task_selector
Registering multi-expert selector...kv_task_selector
Registering multi-expert selector...kv_concat_selector
Registering multi-expert selector...kv_norm_selector
Registering multi-expert selector...kv_concat_norm_selector
Registering multi-expert selector...kv_task_norm_selector


## Create clsuters

In [2]:
hf_api_key = os.environ["HF_TOKEN"]
login(token=hf_api_key)
user = HfApi(token=hf_api_key).whoami()
# hf_repo_id = "sordonia/library-phi_2-v3"
hf_repo_id = "sordonia/library-phi_2-v3-2epc"
local_lib_location = f"/tmp/{hf_repo_id}"
if not os.path.exists(local_lib_location):
    os.makedirs(local_lib_location)
    expert_lib: LocalExpertLibrary = LocalExpertLibrary.create_from_remote(
        HFExpertLibrary(hf_repo_id), local_lib_location
    )
else:
    expert_lib: LocalExpertLibrary = LocalExpertLibrary(local_lib_location)

experts_to_remove = [
    "bool_q_1_0_0",
    "ai2_arc_ARC_Easy_1_0_0",
    "openbookqa_0_1_0",
    "ai2_arc_ARC_Challenge_1_0_0",
    "hellaswag_1_1_0",
    "piqa_1_0_0",
    "winogrande_1_1_0",
]
for expert_name in experts_to_remove:
    if expert_name in expert_lib:
        expert_lib.remove_expert(expert_name)
assert len(expert_lib) == 256

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/v-oostapenko/.cache/huggingface/token
Login successful


In [3]:
def create_embeddings():
    svd_embedder = SVDEmbeddingTransform(
        SVDEmbeddingTransformConfig(sparsity_threshold=0.5)
    )
    embeddings, svd = svd_embedder.transform(expert_lib, upload_to_hf=True)
    del svd_embedder
    return embeddings, svd


embeds = expert_lib.get_auxiliary_data("embeddings")
if len(embeds) == 0:
    print("creating embeddings")
    _, svd = create_embeddings()

# module to embedding
module2embed = {}
for n, m in expert_lib.items():
    module2embed[n] = get_svd_embedding(expert_lib, n)

In [4]:
# Extract the embeddings as a numpy array
embeddings = np.array(list(module2embed.values()))
cosine_sim_matrix = cosine_similarity(embeddings, embeddings)
K = 5 # 10 ,15, 25
kmeans = KMeans(n_clusters=K, init="k-means++", n_init=10, random_state=42)
kmeans.fit(cosine_sim_matrix)
cluster_labels = kmeans.labels_

In [5]:
clusters = defaultdict(list)
# Print the cluster labels for each embedding
for key, label in zip(module2embed.keys(), cluster_labels):
    clusters[label].append(key)

for c, l in clusters.items():
    # print(f"Cluster {c} has {len(l)} elements")
    print(f"c{c}o{K}_2e = {l}")

c0o5_2e = ['glue_sst2_2_0_0', 'adversarial_qa_droberta_generate_question', 'true_case', 'stream_qed', 'cot_esnli', 'cot_sensemaking', 'trec_1_0_0', 'super_glue_wic_1_0_2', 'yelp_polarity_reviews_0_2_0', 'cos_e_v1_11_rationale', 'natural_questions_open_1_0_0', 'web_questions_whats_the_answer', 'anli_r3_0_1_0', 'lambada_1_0_0', 'dream_generate_last_utterance', 'glue_cola_2_0_0', 'ag_news_subset_1_0_0', 'paws_wiki_1_1_0', 'cos_e_v1_11_generate_explanation_given_text', 'cot_creak', 'math_dataset_algebra__linear_1d_1_0_0', 'web_questions_question_answer', 'stream_aqua', 'dbpedia_14_pick_one_category_for_the_following_text', 'dbpedia_14_given_a_choice_of_categories_', 'web_questions_short_general_knowledge_q', 'cnn_dailymail_3_4_0', 'kilt_tasks_hotpotqa_straighforward_qa', 'adversarial_qa_dbidaf_generate_question', 'snli_1_1_0', 'cos_e_v1_11_i_think', 'wiki_hop_original_explain_relation', 'dbpedia_14_given_list_what_category_does_the_paragraph_belong_to', 'fix_punct', 'imdb_reviews_plain_tex

# Save svd file

In [ ]:
hf_api_key = os.environ["HF_TOKEN"]
login(token=hf_api_key)
user = HfApi(token=hf_api_key).whoami()
hf_repo_id = "ostapeno/library-phi_2-v3-10-flan-clusters"
local_lib_location = f"/tmp/{hf_repo_id}"
if not os.path.exists(local_lib_location):
    os.makedirs(local_lib_location)
    expert_lib: LocalExpertLibrary = LocalExpertLibrary.create_from_remote(
        HFExpertLibrary(hf_repo_id), local_lib_location
    )
else:
    expert_lib: LocalExpertLibrary = LocalExpertLibrary(local_lib_location)

In [ ]:
_, svd = create_embeddings()

In [ ]:
# upload embeddings
remote_lib = HFExpertLibrary.from_local(
    expert_lib,
    hf_repo_id,
    force=True,
    upload_aux_data=True,
)

In [ ]:
import pickle
# To save
pickle.dump(svd, open("svd.pkl", "wb"))